# Analisi multiprocessing

Di seguito alcune considerazioni sull'utilizzo
del multiprocessing per la parallelizzazione di
un algoritmo genetico.


## Motivazioni

L'utilizzo del modulo `multiprocessing` è ciò che
viene considerato lo standard per il calcolo
parallelo in Python, soprattutto se si parla di
task CPU bound.

Permette di aggirare il problema introdotto dal
GIL, il quale non permette il classico paradigma
multithread, possibile in altri linguaggi come C o
C++.

Tramite multiprocessing è dunque possibile non
scendere a compromessi in quanto ad espressività
del codice. Il programmatore è libero di scrivere
il suo algoritmo genetico utilizzato tipi e
strutture dati native di Python e, se necessario
può ricorrere a librerie di terze parti senza
problemi.


### Problematiche

Il multiprocessing di contro non è la cosa più
leggera che ci sia. Di certo non è possibile
pensare ad un approccio in cui i processi vengono
creati e distrutti ad ogni iterazione, in quanto
si genererebbe del dell'overhead sicuramente non
trascurabile.

La scelta di creare un pool di processi worker che
rimane in vita dall'inizio alla fine
dell'esecuzione è dunque una scelta quasi obbligata
e che introduce la necessità di un qualche
meccanismo di sincronizzazione per avviare e
mettere in attesa i processi.

L'altro potenziale limite è l'assenza di memoria
condivisa. Non è dunque possibile avere strutture
dati condivise tramite puntatori o definite
globalmente a cui ogni processi può accedere. Anche
qui la scelta di condividere parti della struttura
dati tramite qualche meccanismo di streaming dati
è praticamente obbligata.


#### Memoria condivisa

Esiste la possibilità di creare un blocco di
memoria condivisa che risiede al di fuori di ogni
worker e al quale è possibile accedere in modo
diretto come si farebbe tramite multithreading.

Ho trovato tuttavia questo meccanismo molto
limitante per l'espressività che vorrebbe in
qualche modo garantire la libreria. Il motivo è
che per riuscire a accedere direttamente al blocco
di memoria lo si deve fare tramite oggetti che
supportano il **Buffer Protocol** di Python, come
ad esempio _numpy array_ o _bytearray_.

Questo limita molto i tipi e le strutture dati
che si vogliono impiegare e ci spinge ad un
approccio più classico in cui possiamo manipolare
solo array numerici o simili.

Non sarebbe quindi possibile avere cromosomi
dalla struttura complessa ma soprattutto
personalizzata.


## Modello di calcolo parallelo

Il modello di calcolo proposto si offre di operare
in parallelo nelle fasi di crossover, mutazione e
valutazione.

Una volta selezionati gli individui per la
riproduzione, si suddivide la lista che li contiene
in $W$ chunk uguali, dove $W$ è il numero di
processi worker. Ciascun chunk viene poi inviato
ai worker.

Per la condivisione ho optato per un meccanismo
basato su code (`multiprocessing.Queue`) di
comunicazione. Ogni processo possiede due code, una
per la ricezione dati, l'altra per l'invio.

![queue](images/queue.svg)

Le code permettono di implementare in modo molto
semplice il paradigma _produttore-consumatore_,
fornendo due metodi principali (`put` e `get`)
che permettono rispettivamente di

- Inserire un elemento nella coda. Se questa è
  piena il processo si blocca finché non vi è
  uno slot libero.
- Estrarre un elemento dalla coda. Nel caso in cui
  questa sia vuota ci si blocca in attesa che un
  elemento venga inserito.

Per velocizzare ulteriormente l'invio e la
ricezione dati ho fatto uso della libreria
`asyncio`, la quale, tramite la sintassi
`async`/`await` permette di effettuare operazioni
I/O bound in modo asincrono.


### Pipeline di ricezione

Una prima implementazione consisteva semplicemente
nell'inviare per intero il chunk di struttura dati
al worker. Questo però potrebbe costituire un
ostacolo alle performance nel momento in cui la
fase di calcolo parallelo non è particolarmente
pesante o quando la popolazione presa in esame
risulta particolarmente grande.

Per mitigare il problema è possibile suddividere
ognuno dei chunk da inviare ai worker in ulteriori
_sub-chunk_. In questo modo, il worker potrebbe
ricevere i _sub-chunk_ asincronamente mentre
elabora quelli già ricevuti.

![pipeline](images/recv_pipeline.svg)

Come mostrato in figura, l'estrazione dei chunk di
struttura dati viene delegata ad un thread del
worker. Una volta che il thread estrae un elemento
dalla coda lo mette a sua volta in una coda in
memoria condivisa con il processo worker vero e
proprio. A questo punto il worker può estrarre il
chunk ed iniziare ad elaborarlo mentre il thread
continua ad estrarre il resto dei dati.


### Prestazioni

Per valutare la bontà del modello ho preso in considerazione il tempo
necessario a

- inserire un singolo individuo in coda.
- effettuare le operazioni di crossover, mutazione e valutazione per un singolo
  individuo.
- effettuare le due operazioni precedenti ma per un gruppo di individui.

I test svolti prendono in considerazione popolazioni di $10.000$, $20.000$ e
$50.0000$ individui, tutti identificati da cromosomi composti da $500$ interi.

| cromosoma | individuo (byte) | popolazione (MB) |
| :-------: | :--------------: | :--------------: |
|    100    |       1048       |      10.08       |
|    200    |       1784       |      17.09       |
|    500    |       4344       |      41.51       |


### Operazioni sulla coda

Il primo test è stato effettuato sui tempi medi
di inserimento ed estrazione di un singolo
individuo nella coda. Ho scelto di considerare il
tempo medio e non il peggiore dato che le una
stessa operazione in Python potrebbe impiegare
tempi anche molto differenti. Il tempo più
frequente rimane comunque quello medio.
